In [1]:
from biocrnpyler import ActivatablePromoter, DNAassembly, EnergyTxTlExtract
from biocrnpyler import Compartment, Species
import libsbml
import os
import tempfile

activator = Species("T7RNAP", material_type = "small_molecule")
hill_parameters = {"k":1.0, "n":4, "K":20, "kleak":0.0001}
P_activatable = ActivatablePromoter("P_activtable", activator = activator,
                                    parameters = hill_parameters)
activatable_assembly = DNAassembly("activatable_assembly",
                                   promoter = P_activatable, initial_concentration=10**-3)
E = EnergyTxTlExtract(components=[activatable_assembly],
                        parameter_file="examples/Specialized Tutorials/txtl_toolbox_parameters.txt")
CRN_1 = E.compile_crn()  # compile CRN

new_compartment = Compartment(name="Internal_0")
CRN_2 = E.compile_crn(compartment=new_compartment)  # compile CRN

# assert len(CRN_1.species) == len(CRN_2.species)
# assert len(CRN_1.reactions) == len(CRN_2.reactions) 

# Assert SBML compartments
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as tmp:
    CRN_1.write_sbml_file(tmp.name)
    model = libsbml.readSBMLFromFile(tmp.name).getModel()
    for sbml_species in model.getListOfSpecies():
        assert sbml_species.getCompartment() == "default"
os.remove(tmp.name)

with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as tmp:
    CRN_2.write_sbml_file(tmp.name)
    model = libsbml.readSBMLFromFile(tmp.name).getModel()
    for sbml_species in model.getListOfSpecies():
        assert sbml_species.getCompartment() == "Internal_0"
os.remove(tmp.name)
        

c:\Users\ayush\Box\Research\bioCRNpyler\biocrnpyler\core\parameter.py:507: UserWarning: parameter file contains no unit column! Please add a column named ['unit', 'units'].
  warn(f"parameter file contains no {accepted_name} column! Please add a "


In [2]:
CRN_1.species

[protein_Ribo,
 dna_activatable_assembly,
 protein_RNAase,
 metabolite_amino_acids,
 rna_activatable_assembly,
 metabolite_Fuel_3PGA,
 protein_RNAP,
 metabolite_NDPs,
 small_molecule_T7RNAP,
 metabolite_NTPs,
 complex_protein_RNAase_rna_activatable_assembly_]

In [3]:
CRN_2.species

[rna_activatable_assembly_Internal_0,
 metabolite_NTPs_Internal_0,
 complex_protein_RNAase_rna_activatable_assembly_Internal_0_,
 protein_Ribo_Internal_0,
 metabolite_amino_acids_Internal_0,
 metabolite_Fuel_3PGA_Internal_0,
 protein_RNAase_Internal_0,
 protein_RNAP_Internal_0,
 dna_activatable_assembly_Internal_0,
 metabolite_NDPs_Internal_0,
 small_molecule_T7RNAP_Internal_0]

In [4]:
hill_parameters = {"k":1.0, "n":4, "K":20, "kleak":0.0001}
P_activatable = ActivatablePromoter("P_activtable", activator = "activator",
                                    parameters = hill_parameters)
activatable_assembly = DNAassembly("activatable_assembly",
                                promoter = P_activatable)
E = EnergyTxTlExtract(components=[activatable_assembly],
                        parameter_file="examples/Specialized Tutorials/txtl_toolbox_parameters.txt")
CRN_1 = E.compile_crn()  # compile CRN

new_compartment = Compartment(name="Internal_0")
CRN_2 = E.compile_crn(compartment=new_compartment)  # compile CRN

assert len(CRN_1.species) == len(CRN_2.species)
assert len(CRN_1.reactions) == len(CRN_2.reactions) 

# Assert SBML compartments
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as tmp:
    CRN_1.write_sbml_file(tmp.name)
    model = libsbml.readSBMLFromFile(tmp.name).getModel()
    for sbml_species in model.getListOfSpecies():
        assert sbml_species.getCompartment() == "default"
os.remove(tmp.name)

with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as tmp:
    CRN_2.write_sbml_file(tmp.name)
    model = libsbml.readSBMLFromFile(tmp.name).getModel()
    for sbml_species in model.getListOfSpecies():
        assert sbml_species.getCompartment() == "Internal_0"
os.remove(tmp.name)

c:\Users\ayush\Box\Research\bioCRNpyler\biocrnpyler\core\parameter.py:507: UserWarning: parameter file contains no unit column! Please add a column named ['unit', 'units'].
  warn(f"parameter file contains no {accepted_name} column! Please add a "


In [1]:
from biocrnpyler import ChemicalReactionNetwork, Species
S_1 = Species("S_1")
S_2 = Species("S_2")
CRN = ChemicalReactionNetwork([S_1], [])

#Internal species copied correctly to return
assert S_1 in CRN.species

#assert species are copied to internal species set
assert not S_1 is list(CRN._species_set)[0]

#Returned list does not affect internal species
CRN.species[0] = S_2
assert S_2 not in CRN.species

#add species affects internal species list
CRN.add_species(S_2)
assert S_2 in CRN.species
#assert correct copying
assert S_2 is not list(CRN._species_set)[1]



#Test bypassing species protection
CRN = ChemicalReactionNetwork([], [])
CRN.add_species([S_1], copy_species = False)
assert S_1 is list(CRN._species_set)[0]

In [2]:
# check that this error is raised when trying to modify species
from unittest import TestCase

tc = TestCase()
with tc.assertRaisesRegex(AttributeError, "The species in a CRN cannot be removed or modified"):
    CRN.species = []

In [5]:
CRN.species[0] = S_2

In [12]:
from biocrnpyler import ChemicalComplex, ParameterKey, ExpressionExtract
from biocrnpyler import parameter_to_value

S1 = Species("S1")
S2 = Species("S2")

C = ChemicalComplex([S1, S2], name = "C")
S3 = C.get_species()

mixture_name = "M"
key1 = ParameterKey(mechanism = "initial concentration", part_id = mixture_name, name = str(S1))
key2 = ParameterKey(mechanism = "initial concentration", part_id = mixture_name, name = str(S2))
key3 = ParameterKey(mechanism = "initial concentration", part_id = mixture_name, name = str(S3))

C = ChemicalComplex([S1, S2], name = "C", parameters = {key1:.11, key2:.22}, initial_concentration = .33)
M = ExpressionExtract(name = mixture_name, components = [C], parameters = {key1:1.1, key2:2.2, key3:3.3, "kb":1.0, "ku":1.0})

#Initial condition found under the Species name, in the Component, not Mixture
CRN = M.compile_crn()
#Initial condition found under the Species name, in the Component, not Mixture
# assert parameter_to_value(CRN.initial_concentration_dict[S1]) == .11
# assert parameter_to_value(CRN.initial_concentration_dict[S2]) == .22
# assert parameter_to_value(CRN.initial_concentration_dict[S3]) == .33

In [16]:
parameter_to_value(CRN.initial_concentration_dict[S2])

2.2